In [2]:
import os


relative_train_path = "train"
current_dir = os.getcwd()  #Here
train_path = os.path.join(current_dir, relative_train_path)

train_files = os.listdir(train_path)

train_files[:10]

['apple_49.xml',
 'apple_75.xml',
 'apple_61.xml',
 'apple_75.jpg',
 'apple_61.jpg',
 'apple_49.jpg',
 'orange_3.jpg',
 'orange_3.xml',
 'orange_28.jpg',
 'banana_50.jpg']

In [3]:
import os


relative_test_path = "test"
current_dir = os.getcwd()  #Here
test_path = os.path.join(current_dir, relative_test_path)

test_files = os.listdir(test_path)

test_files[:10]

['banana_93.xml',
 'banana_78.jpg',
 'banana_87.xml',
 'banana_93.jpg',
 'banana_87.jpg',
 'banana_78.xml',
 'banana_86.jpg',
 'banana_79.xml',
 'banana_92.jpg',
 'banana_79.jpg']

In [4]:
import cv2
from tqdm import tqdm
import xml.etree.ElementTree as ET



def convert_to_yolo_format(xml_file, img_path, class_mapping):
    """Convert XML annotation to YOLO format annotation."""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get image dimensions
    img = cv2.imread(img_path)
    img_height, img_width = img.shape[:2]

    yolo_data = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in class_mapping:
            continue

        class_index = class_mapping[class_name]
        bndbox = obj.find('bndbox')
        
        # Extract bounding box coordinates
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        # Convert to YOLO format
        x_center = (xmin + xmax) / 2.0
        y_center = (ymin + ymax) / 2.0
        width = xmax - xmin
        height = ymax - ymin

        # Normalize by image dimensions
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height

        yolo_data.append((class_index, x_center, y_center, width, height))

    return yolo_data

# Define class mapping
classes = ["apple", "banana", "orange"]
class_mapping = {class_name: idx for idx, class_name in enumerate(classes)}

# Convert train dataset
yolo_train_annotations = {}
for file in tqdm(train_files):
    if file.endswith(".jpg"):
        img_path = os.path.join(train_path, file)
        xml_file = os.path.join(train_path, file.replace('.jpg', '.xml'))

        yolo_data = convert_to_yolo_format(xml_file, img_path, class_mapping)
        yolo_train_annotations[img_path] = yolo_data

# Convert test dataset
yolo_test_annotations = {}
for file in tqdm(test_files):
    if file.endswith(".jpg"):
        img_path = os.path.join(test_path, file)
        xml_file = os.path.join(test_path, file.replace('.jpg', '.xml'))

        yolo_data = convert_to_yolo_format(xml_file, img_path, class_mapping)
        yolo_test_annotations[img_path] = yolo_data

len(yolo_train_annotations), len(yolo_test_annotations)

 91%|█████████ | 437/480 [00:00<00:00, 544.70it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
100%|██████████| 120/120 [00:00<00:00, 553.48it/s]


(240, 60)

In [5]:
from PIL import Image
import os

def save_yolo_format(yolo_annotations, save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for img_path, annotations in yolo_annotations.items():
        # Load the image using Pillow and convert to RGB
        image = Image.open(img_path).convert('RGB')
        
        # Save the RGB image to the new directory
        new_img_path = os.path.join(save_path, os.path.basename(img_path))
        image.save(new_img_path)

        # Write the annotations to a new .txt file
        txt_path = os.path.join(save_path, os.path.basename(img_path).replace('.jpg', '.txt'))
        with open(txt_path, 'w') as f:
            for annotation in annotations:
                f.write(' '.join(map(str, annotation)) + '\n')

# 훈련 및 테스트 어노테이션 저장
yolo_dp_train_path = os.path.join(current_dir, "yolo_dp_train")
yolo_dp_test_path = os.path.join(current_dir, "yolo_dp_test")

save_yolo_format(yolo_train_annotations, yolo_dp_train_path)
save_yolo_format(yolo_test_annotations, yolo_dp_test_path)


/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [6]:
data_config = """
train: /Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolo_dp_train
val: /Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolo_dp_test

nc: 3  # number of classes
names: ['apple', 'banana', 'orange']  # class names
"""

with open("/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolov5/data/data_config.yaml", 'w') as f:
    f.write(data_config)

In [7]:
!python /Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolov5/train.py --img 640 --batch 16 --epochs 1 --data /Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolov5/data/data_config.yaml --cfg /Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolov5/models/yolov5s.yaml --weights /Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolov5/yolov5s.pt

train: weights=/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolov5/yolov5s.pt, cfg=/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolov5/models/yolov5s.yaml, data=/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/yolov5/data/data_config.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
^C

YOLOv5 🚀 v7.0-207-gdf48c20 Python-3.11.4 torch-2.0.1 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=

# Show example

In [8]:
# yolo_test_annotations의 키들을 출력합니다.
list(yolo_test_annotations.keys())[:10]  # 처음 10개만 출력합니다.


['/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_78.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_93.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_87.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_86.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_92.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_79.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/apple_89.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_84.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_90.jpg',
 '/Users/owo/HOUSE/@Code/@Project/ID_FruitDetection/test/banana_91.jpg']

In [10]:
print(selected_test_images)

NameError: name 'selected_test_images' is not defined

In [12]:
import numpy as np
import random
from PIL import Image, ImageDraw
from pathlib import Path
import matplotlib.pyplot as plt
import torch

# 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp3/weights/best.pt')
model.eval()

def draw_boxes(image_path, boxes, labels=None, color='red'):
    with Image.open(image_path) as img:
        draw = ImageDraw.Draw(img)
        for i, box in enumerate(boxes):
            xyxy = box[:4]
            label = f"{labels[i]}" if labels else ""
            draw.rectangle(xyxy, outline=color, width=2)
            draw.text((xyxy[0], xyxy[1] - 10), label, fill=color)
    return img

def predict_and_show(images, annotations):
    fig, axs = plt.subplots(3, len(images), figsize=(20, 12))
    
    # 원본 이미지 표시
    for i, img_path in enumerate(images):
        with Image.open(img_path) as im:
            axs[0, i].imshow(im)
            axs[0, i].set_title("Original")
            axs[0, i].axis('off')

    # 원본 YOLO 어노테이션 그리기
    for i, img_path in enumerate(images):
        boxes = annotations[img_path]['boxes']
        labels = annotations[img_path]['labels']
        img = draw_boxes(img_path, boxes, labels)
        axs[1, i].imshow(img)
        axs[1, i].set_title("Original YOLO Annotations")
        axs[1, i].axis('off')

    # 예측된 바운딩 박스 그리기
    results = model(images)
    for i, img_path in enumerate(images):
        pred_boxes = results.pred[i][:, :-1].tolist()
        pred_labels = [model.names[int(cls)] for cls in results.pred[i][:, -1]]
        img = draw_boxes(img_path, pred_boxes, pred_labels)
        axs[2, i].imshow(img)
        axs[2, i].set_title("Predicted")
        axs[2, i].axis('off')
    
    plt.tight_layout()
    plt.show()

# 이미지 경로를 Path 객체로 가져옵니다.
test_images = [Path(p) for p in os.listdir(yolo_dp_test_path) if p.endswith('.jpg')]

# 테스트 이미지 중에서 무작위로 10개를 선택합니다.
selected_test_images = random.sample(test_images, 10)

# 선택된 이미지에 대한 어노테이션을 추출하기 위해 테스트 이미지 경로를 조정합니다.
corrected_test_images = [img_path.replace("yolo_dp_test", "test") for img_path in selected_test_images]

# 선택된 이미지에 대한 어노테이션 추출
selected_annotations = {
    str(img_path): {  
        'boxes': [ann[:4] for ann in yolo_test_annotations[str(img_path)]],
        'labels': [model.names[int(ann[4])] for ann in yolo_test_annotations[str(img_path)]]
    }
    for img_path in corrected_test_images
}

# 예측 및 결과 출력
predict_and_show(corrected_test_images, selected_annotations)

Using cache found in /Users/owo/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-8-9 Python-3.11.4 torch-2.0.1 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


TypeError: Path.replace() takes 2 positional arguments but 3 were given